In [1]:
import pandas as pd
import polars as pl

In [2]:
# chunk_size = 100000  # Number of rows per chunk
df = pl.read_csv('US_Accidents_March23.csv') #, chunksize = chunk_size)

In [3]:
df.head(5)

ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
str,str,i64,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str
"""A-1""","""Source2""",3,"""2016-02-08 05:46:00""","""2016-02-08 11:00:00""",39.865147,-84.058723,null,null,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US""","""US/Eastern""","""KFFO""","""2016-02-08 05:58:00""",36.9,null,91.0,29.68,10.0,"""Calm""",null,0.02,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Night"""
"""A-2""","""Source2""",2,"""2016-02-08 06:07:59""","""2016-02-08 06:37:59""",39.928059,-82.831184,null,null,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US""","""US/Eastern""","""KCMH""","""2016-02-08 05:51:00""",37.9,null,100.0,29.65,10.0,"""Calm""",null,0.0,"""Light Rain""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Night""","""Night""","""Day"""
"""A-3""","""Source2""",2,"""2016-02-08 06:49:27""","""2016-02-08 07:19:27""",39.063148,-84.032608,null,null,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""","""2016-02-08 06:56:00""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,null,"""Overcast""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Night""","""Night""","""Day""","""Day"""
"""A-4""","""Source2""",3,"""2016-02-08 07:23:34""","""2016-02-08 07:53:34""",39.747753,-84.205582,null,null,0.01,"""Accident on I-75 Southbound at…","""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US""","""US/Eastern""","""KDAY""","""2016-02-08 07:38:00""",35.1,31.0,96.0,29.64,9.0,"""SW""",4.6,null,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,false,false,"""Night""","""Day""","""Day""","""Day"""
"""A-5""","""Source2""",2,"""2016-02-08 07:39:07""","""2016-02-08 08:09:07""",39.627781,-84.188354,null,null,0.01,"""Accident on McEwen Rd at OH-72…","""Miamisburg Centerville Rd""","""Dayton""","""Montgomery""","""OH""","""45459""","""US""","""US/Eastern""","""KMGY""","""2016-02-08 07:53:00""",36.0,33.3,89.0,29.65,6.0,"""SW""",3.5,null,"""Mostly Cloudy""",false,false,false,false,false,false,false,false,false,false,false,true,false,"""Day""","""Day""","""Day""","""Day"""


In [4]:
df.height

7728394

In [5]:
df.width

46

In [6]:
df.columns

['ID',
 'Source',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Country',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Weather_Condition',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight']

In [7]:
#Checking how many values are false in the boolean columns
cols = ['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop']
false_counts = df.select([(pl.col(col) == False).sum().alias(col) for col in cols])
for col in cols:
    print(f"False entries in {col} : {false_counts[0, col]}")

False entries in Amenity : 7632060
False entries in Bump : 7724880
False entries in Crossing : 6854631
False entries in Give_Way : 7691812
False entries in Junction : 7157052
False entries in No_Exit : 7708849
False entries in Railway : 7661415
False entries in Roundabout : 7728145
False entries in Station : 7526493
False entries in Stop : 7514023
False entries in Traffic_Calming : 7720796
False entries in Traffic_Signal : 6584622
False entries in Turning_Loop : 7728394


In [8]:
#The 10 columns that are almost entirely false and thus not useful
df_drop = df.drop(['Amenity','Bump','Give_Way','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Turning_Loop'])
df_drop.width

36

In [9]:
#Just a couple of extra columns that do not give us enough information on their own to warrant staying.
#Closely associated with Sunset, so they will not add to our analysis 
df_drop = df_drop.drop(['Nautical_Twilight','Astronomical_Twilight', 'Civil_Twilight']) 

In [10]:
df_drop.width

33

In [11]:
df_drop.columns

['ID',
 'Source',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Country',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Weather_Condition',
 'Crossing',
 'Junction',
 'Traffic_Signal',
 'Sunrise_Sunset']

In [12]:
cols = ['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Street',
        'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)','Humidity(%)'
        , 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Crossing', 'Junction',
        'Traffic_Signal', 'Sunrise_Sunset']
for col in cols:
    null_count = df_drop.select(pl.col(col).null_count())[0, 0]  # Get the count of nulls in the column
    print(f"Null entries in {col} : {null_count}")

Null entries in ID : 0
Null entries in Source : 0
Null entries in Severity : 0
Null entries in Start_Time : 0
Null entries in End_Time : 0
Null entries in Start_Lat : 0
Null entries in Start_Lng : 0
Null entries in End_Lat : 3402762
Null entries in End_Lng : 3402762
Null entries in Distance(mi) : 0
Null entries in Description : 5
Null entries in Street : 10869
Null entries in City : 253
Null entries in County : 0
Null entries in State : 0
Null entries in Zipcode : 1915
Null entries in Country : 0
Null entries in Timezone : 7808
Null entries in Airport_Code : 22635
Null entries in Weather_Timestamp : 120228
Null entries in Temperature(F) : 163853
Null entries in Wind_Chill(F) : 1999019
Null entries in Humidity(%) : 174144
Null entries in Pressure(in) : 140679
Null entries in Visibility(mi) : 177098
Null entries in Wind_Direction : 175206
Null entries in Wind_Speed(mph) : 571233
Null entries in Precipitation(in) : 2203586
Null entries in Weather_Condition : 173459
Null entries in Crossin

In [13]:
df_drop = df_drop.drop(['End_Lat','End_Lng']) #since majority of the values are null and these columns are insignificant

In [14]:
df_drop.width

31

In [15]:
bool_cols = ['Junction', 'Crossing', 'Traffic_Signal']
df_drop = df_drop.with_columns([pl.col(col).cast(pl.Int32).alias(col) for col in bool_cols])

In [16]:
df_drop.head(3)

ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Crossing,Junction,Traffic_Signal,Sunrise_Sunset
str,str,i64,str,str,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,i32,i32,i32,str
"""A-1""","""Source2""",3,"""2016-02-08 05:46:00""","""2016-02-08 11:00:00""",39.865147,-84.058723,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US""","""US/Eastern""","""KFFO""","""2016-02-08 05:58:00""",36.9,null,91.0,29.68,10.0,"""Calm""",null,0.02,"""Light Rain""",0,0,0,"""Night"""
"""A-2""","""Source2""",2,"""2016-02-08 06:07:59""","""2016-02-08 06:37:59""",39.928059,-82.831184,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US""","""US/Eastern""","""KCMH""","""2016-02-08 05:51:00""",37.9,null,100.0,29.65,10.0,"""Calm""",null,0.0,"""Light Rain""",0,0,0,"""Night"""
"""A-3""","""Source2""",2,"""2016-02-08 06:49:27""","""2016-02-08 07:19:27""",39.063148,-84.032608,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US""","""US/Eastern""","""KI69""","""2016-02-08 06:56:00""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,null,"""Overcast""",0,0,1,"""Night"""


In [17]:
for col in df_drop.columns:
    unique_count = df_drop.select(pl.col(col).n_unique())[0, 0]
    print(f"Unique entries in {col} : {unique_count}")

Unique entries in ID : 7728394
Unique entries in Source : 3
Unique entries in Severity : 4
Unique entries in Start_Time : 6131796
Unique entries in End_Time : 6705355
Unique entries in Start_Lat : 2449172
Unique entries in Start_Lng : 2493905
Unique entries in Distance(mi) : 22382
Unique entries in Description : 3761579
Unique entries in Street : 336307
Unique entries in City : 13679
Unique entries in County : 1871
Unique entries in State : 49
Unique entries in Zipcode : 825095
Unique entries in Country : 1
Unique entries in Timezone : 5
Unique entries in Airport_Code : 2046
Unique entries in Weather_Timestamp : 941332
Unique entries in Temperature(F) : 861
Unique entries in Wind_Chill(F) : 1002
Unique entries in Humidity(%) : 101
Unique entries in Pressure(in) : 1145
Unique entries in Visibility(mi) : 93
Unique entries in Wind_Direction : 25
Unique entries in Wind_Speed(mph) : 185
Unique entries in Precipitation(in) : 300
Unique entries in Weather_Condition : 145
Unique entries in Cro

In [18]:
df_drop = df_drop.drop(['Country']) #This is a uniform column displaying just the 'US'

In [19]:
df_drop.width

30

In [20]:
df_drop = df_drop.drop(['Airport_Code','ID']) #not useful in our context

In [21]:
df_drop.width

28

In [22]:
df_drop = df_drop.drop(['Weather_Timestamp']) #This relates to what time the weather data was recorded, so irrelevant to us

In [23]:
df_drop.columns

['Source',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Timezone',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Weather_Condition',
 'Crossing',
 'Junction',
 'Traffic_Signal',
 'Sunrise_Sunset']

In [24]:
# List of columns to fill null values with mean
cols_to_fill = ['Wind_Chill(F)', 'Wind_Speed(mph)', 'Visibility(mi)', 'Precipitation(in)', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)']

# Fill null values with the mean of the column
df_drop = df_drop.with_columns([pl.col(col).fill_null(pl.col(col).mean()).alias(col) for col in cols_to_fill])
df_drop.head(10)

Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Crossing,Junction,Traffic_Signal,Sunrise_Sunset
str,i64,str,str,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,i32,i32,i32,str
"""Source2""",3,"""2016-02-08 05:46:00""","""2016-02-08 11:00:00""",39.865147,-84.058723,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US/Eastern""",36.9,58.251048,91.0,29.68,10.0,"""Calm""",7.68549,0.02,"""Light Rain""",0,0,0,"""Night"""
"""Source2""",2,"""2016-02-08 06:07:59""","""2016-02-08 06:37:59""",39.928059,-82.831184,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US/Eastern""",37.9,58.251048,100.0,29.65,10.0,"""Calm""",7.68549,0.0,"""Light Rain""",0,0,0,"""Night"""
"""Source2""",2,"""2016-02-08 06:49:27""","""2016-02-08 07:19:27""",39.063148,-84.032608,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US/Eastern""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,0.008407,"""Overcast""",0,0,1,"""Night"""
"""Source2""",3,"""2016-02-08 07:23:34""","""2016-02-08 07:53:34""",39.747753,-84.205582,0.01,"""Accident on I-75 Southbound at…","""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US/Eastern""",35.1,31.0,96.0,29.64,9.0,"""SW""",4.6,0.008407,"""Mostly Cloudy""",0,0,0,"""Night"""
"""Source2""",2,"""2016-02-08 07:39:07""","""2016-02-08 08:09:07""",39.627781,-84.188354,0.01,"""Accident on McEwen Rd at OH-72…","""Miamisburg Centerville Rd""","""Dayton""","""Montgomery""","""OH""","""45459""","""US/Eastern""",36.0,33.3,89.0,29.65,6.0,"""SW""",3.5,0.008407,"""Mostly Cloudy""",0,0,1,"""Day"""
"""Source2""",3,"""2016-02-08 07:44:26""","""2016-02-08 08:14:26""",40.10059,-82.925194,0.01,"""Accident on I-270 Outerbelt No…","""Westerville Rd""","""Westerville""","""Franklin""","""OH""","""43081""","""US/Eastern""",37.9,35.5,97.0,29.63,7.0,"""SSW""",3.5,0.03,"""Light Rain""",0,0,0,"""Day"""
"""Source2""",2,"""2016-02-08 07:59:35""","""2016-02-08 08:29:35""",39.758274,-84.230507,0.0,"""Accident on Oakridge Dr at Woo…","""N Woodward Ave""","""Dayton""","""Montgomery""","""OH""","""45417-2476""","""US/Eastern""",34.0,31.0,100.0,29.66,7.0,"""WSW""",3.5,0.008407,"""Overcast""",0,0,0,"""Day"""
"""Source2""",3,"""2016-02-08 07:59:58""","""2016-02-08 08:29:58""",39.770382,-84.194901,0.01,"""Accident on I-75 Southbound at…","""N Main St""","""Dayton""","""Montgomery""","""OH""","""45405""","""US/Eastern""",34.0,31.0,100.0,29.66,7.0,"""WSW""",3.5,0.008407,"""Overcast""",0,0,0,"""Day"""
"""Source2""",2,"""2016-02-08 08:00:40""","""2016-02-08 08:30:40""",39.778061,-84.172005,0.0,"""Accident on Notre Dame Ave at …","""Notre Dame Ave""","""Dayton""","""Montgomery""","""OH""","""45404-1923""","""US/Eastern""",33.3,58.251048,99.0,29.67,5.0,"""SW""",1.2,0.008407,"""Mostly Cloudy""",0,0,0,"""Day"""


In [25]:
cols = ['Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Description', 'Street','City', 'County', 'State', 
        'Zipcode', 'Timezone', 'Temperature(F)', 'Wind_Chill(F)','Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Direction', 'Wind_Speed(mph)',
        'Precipitation(in)', 'Weather_Condition', 'Crossing', 'Junction','Traffic_Signal', 'Sunrise_Sunset']
for col in cols:
    null_count = df_drop.select(pl.col(col).null_count())[0, 0]  # Get the count of nulls in the column
    print(f"Null entries in {col} : {null_count}")

Null entries in Source : 0
Null entries in Severity : 0
Null entries in Start_Time : 0
Null entries in End_Time : 0
Null entries in Start_Lat : 0
Null entries in Start_Lng : 0
Null entries in Distance(mi) : 0
Null entries in Description : 5
Null entries in Street : 10869
Null entries in City : 253
Null entries in County : 0
Null entries in State : 0
Null entries in Zipcode : 1915
Null entries in Timezone : 7808
Null entries in Temperature(F) : 0
Null entries in Wind_Chill(F) : 0
Null entries in Humidity(%) : 0
Null entries in Pressure(in) : 0
Null entries in Visibility(mi) : 0
Null entries in Wind_Direction : 175206
Null entries in Wind_Speed(mph) : 0
Null entries in Precipitation(in) : 0
Null entries in Weather_Condition : 173459
Null entries in Crossing : 0
Null entries in Junction : 0
Null entries in Traffic_Signal : 0
Null entries in Sunrise_Sunset : 23246


In [26]:
df_drop.head(3)

Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Crossing,Junction,Traffic_Signal,Sunrise_Sunset
str,i64,str,str,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,i32,i32,i32,str
"""Source2""",3,"""2016-02-08 05:46:00""","""2016-02-08 11:00:00""",39.865147,-84.058723,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US/Eastern""",36.9,58.251048,91.0,29.68,10.0,"""Calm""",7.68549,0.02,"""Light Rain""",0,0,0,"""Night"""
"""Source2""",2,"""2016-02-08 06:07:59""","""2016-02-08 06:37:59""",39.928059,-82.831184,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US/Eastern""",37.9,58.251048,100.0,29.65,10.0,"""Calm""",7.68549,0.0,"""Light Rain""",0,0,0,"""Night"""
"""Source2""",2,"""2016-02-08 06:49:27""","""2016-02-08 07:19:27""",39.063148,-84.032608,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US/Eastern""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,0.008407,"""Overcast""",0,0,1,"""Night"""


In [27]:
# Step 1: Clean the datetime strings to remove fractional seconds
df_drop = df_drop.with_columns([
    pl.col('Start_Time').str.replace(r'\.\d+', '').alias('Start_Time'),  # Remove anything after the seconds
    pl.col('End_Time').str.replace(r'\.\d+', '').alias('End_Time')      # Remove anything after the seconds
])

# Step 2: Identify invalid datetime entries (optional but useful for debugging)
invalid_start = df_drop.filter(~pl.col('Start_Time').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=True).is_not_null())
invalid_end = df_drop.filter(~pl.col('End_Time').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=True).is_not_null())

print("Invalid Start_Time entries:")
print(invalid_start)

print("Invalid End_Time entries:")
print(invalid_end)

# Step 3: Convert the cleaned time data into workable DateTime objects with strict=False
df_drop = df_drop.with_columns([
    pl.col('Start_Time').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=False).alias('Start_Time'),
    pl.col('End_Time').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S', strict=False).alias('End_Time')
])

# Breaking the start time into usable independent features
df_drop = df_drop.with_columns([
    pl.col('Start_Time').dt.year().alias('Year'),
    pl.col('Start_Time').dt.month().alias('Month'),
    pl.col('Start_Time').dt.day().alias('Day'),
    pl.col('Start_Time').dt.hour().alias('Time_S'),
    pl.col('Start_Time').dt.weekday().alias('Weekday'),
    (pl.col('End_Time') - pl.col('Start_Time')).dt.total_seconds().alias('Duration')
])

# Drop the original Start_Time and End_Time columns
df_drop = df_drop.drop(['Start_Time', 'End_Time'])

Invalid Start_Time entries:
shape: (0, 27)
┌────────┬──────────┬────────────┬──────────┬───┬──────────┬──────────┬────────────────┬────────────────┐
│ Source ┆ Severity ┆ Start_Time ┆ End_Time ┆ … ┆ Crossing ┆ Junction ┆ Traffic_Signal ┆ Sunrise_Sunset │
│ ---    ┆ ---      ┆ ---        ┆ ---      ┆   ┆ ---      ┆ ---      ┆ ---            ┆ ---            │
│ str    ┆ i64      ┆ str        ┆ str      ┆   ┆ i32      ┆ i32      ┆ i32            ┆ str            │
╞════════╪══════════╪════════════╪══════════╪═══╪══════════╪══════════╪════════════════╪════════════════╡
└────────┴──────────┴────────────┴──────────┴───┴──────────┴──────────┴────────────────┴────────────────┘
Invalid End_Time entries:
shape: (0, 27)
┌────────┬──────────┬────────────┬──────────┬───┬──────────┬──────────┬────────────────┬────────────────┐
│ Source ┆ Severity ┆ Start_Time ┆ End_Time ┆ … ┆ Crossing ┆ Junction ┆ Traffic_Signal ┆ Sunrise_Sunset │
│ ---    ┆ ---      ┆ ---        ┆ ---      ┆   ┆ ---      ┆ ---    

In [28]:
df_drop.head(15)

Source,Severity,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Crossing,Junction,Traffic_Signal,Sunrise_Sunset,Year,Month,Day,Time_S,Weekday,Duration
str,i64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,i32,i32,i32,str,i32,i8,i8,i8,i8,i64
"""Source2""",3,39.865147,-84.058723,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US/Eastern""",36.9,58.251048,91.0,29.68,10.0,"""Calm""",7.68549,0.02,"""Light Rain""",0,0,0,"""Night""",2016,2,8,5,1,18840
"""Source2""",2,39.928059,-82.831184,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US/Eastern""",37.9,58.251048,100.0,29.65,10.0,"""Calm""",7.68549,0.0,"""Light Rain""",0,0,0,"""Night""",2016,2,8,6,1,1800
"""Source2""",2,39.063148,-84.032608,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US/Eastern""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,0.008407,"""Overcast""",0,0,1,"""Night""",2016,2,8,6,1,1800
"""Source2""",3,39.747753,-84.205582,0.01,"""Accident on I-75 Southbound at…","""I-75 S""","""Dayton""","""Montgomery""","""OH""","""45417""","""US/Eastern""",35.1,31.0,96.0,29.64,9.0,"""SW""",4.6,0.008407,"""Mostly Cloudy""",0,0,0,"""Night""",2016,2,8,7,1,1800
"""Source2""",2,39.627781,-84.188354,0.01,"""Accident on McEwen Rd at OH-72…","""Miamisburg Centerville Rd""","""Dayton""","""Montgomery""","""OH""","""45459""","""US/Eastern""",36.0,33.3,89.0,29.65,6.0,"""SW""",3.5,0.008407,"""Mostly Cloudy""",0,0,1,"""Day""",2016,2,8,7,1,1800
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Source2""",3,39.952812,-83.119293,0.01,"""Accident on I-270 Outerbelt No…","""Outerbelt S""","""Columbus""","""Franklin""","""OH""","""43228""","""US/Eastern""",35.6,30.7,93.0,29.64,5.0,"""WNW""",5.8,0.008407,"""Rain""",1,1,0,"""Day""",2016,2,8,8,1,1800
"""Source2""",3,39.932709,-82.83091,0.01,"""One lane blocked due to accide…","""I-70 E""","""Reynoldsburg""","""Franklin""","""OH""","""43068""","""US/Eastern""",37.4,33.8,100.0,29.62,3.0,"""SSW""",4.6,0.02,"""Light Rain""",0,0,0,"""Day""",2016,2,8,8,1,1800
"""Source2""",2,39.737633,-84.149933,0.0,"""Accident on Revere Ave at Wate…","""Watervliet Ave""","""Dayton""","""Montgomery""","""OH""","""45420-1863""","""US/Eastern""",33.8,58.251048,100.0,29.63,3.0,"""SW""",2.3,0.008407,"""Overcast""",0,0,0,"""Day""",2016,2,8,8,1,1800


In [29]:
cols = ['Source', 'Severity', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Description', 'Street','City', 'County', 'State', 'Zipcode', 'Timezone', 
        'Temperature(F)', 'Wind_Chill(F)','Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Direction', 'Wind_Speed(mph)','Precipitation(in)', 
        'Weather_Condition', 'Crossing', 'Junction','Traffic_Signal', 'Sunrise_Sunset', 'Year', 'Month', 'Day', 'Time_S', 'Weekday', 'Duration']
for col in cols:
    null_count = df_drop.select(pl.col(col).null_count())[0, 0]  # Get the count of nulls in the column
    print(f"Null entries in {col} : {null_count}")

Null entries in Source : 0
Null entries in Severity : 0
Null entries in Start_Lat : 0
Null entries in Start_Lng : 0
Null entries in Distance(mi) : 0
Null entries in Description : 5
Null entries in Street : 10869
Null entries in City : 253
Null entries in County : 0
Null entries in State : 0
Null entries in Zipcode : 1915
Null entries in Timezone : 7808
Null entries in Temperature(F) : 0
Null entries in Wind_Chill(F) : 0
Null entries in Humidity(%) : 0
Null entries in Pressure(in) : 0
Null entries in Visibility(mi) : 0
Null entries in Wind_Direction : 175206
Null entries in Wind_Speed(mph) : 0
Null entries in Precipitation(in) : 0
Null entries in Weather_Condition : 173459
Null entries in Crossing : 0
Null entries in Junction : 0
Null entries in Traffic_Signal : 0
Null entries in Sunrise_Sunset : 23246
Null entries in Year : 0
Null entries in Month : 0
Null entries in Day : 0
Null entries in Time_S : 0
Null entries in Weekday : 0
Null entries in Duration : 0


In [30]:
df_drop.width

31

In [31]:
df_drop.height

7728394

In [32]:
# Filtering out rows where 'Weather_Condition' or 'Wind_Direction' are null
df_drop = df_drop.filter(~pl.col('Weather_Condition').is_null() | ~pl.col('Wind_Direction').is_null())

In [33]:
df_drop.height

7593604

In [34]:
unique_values = df_drop["Wind_Direction"].unique()
print(unique_values.to_list())

['SE', 'SSW', 'NE', 'Calm', 'W', 'S', 'North', 'E', 'NNW', 'N', 'NW', 'East', 'WNW', 'CALM', 'WSW', 'VAR', 'West', 'South', 'NNE', 'ENE', 'SW', 'SSE', 'ESE', 'Variable', None]


In [35]:
df_drop.head(3)

Source,Severity,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Crossing,Junction,Traffic_Signal,Sunrise_Sunset,Year,Month,Day,Time_S,Weekday,Duration
str,i64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,i32,i32,i32,str,i32,i8,i8,i8,i8,i64
"""Source2""",3,39.865147,-84.058723,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US/Eastern""",36.9,58.251048,91.0,29.68,10.0,"""Calm""",7.68549,0.02,"""Light Rain""",0,0,0,"""Night""",2016,2,8,5,1,18840
"""Source2""",2,39.928059,-82.831184,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US/Eastern""",37.9,58.251048,100.0,29.65,10.0,"""Calm""",7.68549,0.0,"""Light Rain""",0,0,0,"""Night""",2016,2,8,6,1,1800
"""Source2""",2,39.063148,-84.032608,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US/Eastern""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,0.008407,"""Overcast""",0,0,1,"""Night""",2016,2,8,6,1,1800


In [36]:
#as we can see S means the same as South and N means the same as North and so on, hence cleaning this up
df_drop = df_drop.with_columns(
    pl.when(pl.col('Wind_Direction') == 'Calm').then(pl.lit('CALM'))
    .when(pl.col('Wind_Direction') == 'North').then(pl.lit('N'))
    .when(pl.col('Wind_Direction') == 'South').then(pl.lit('S'))
    .when(pl.col('Wind_Direction') == 'East').then(pl.lit('E'))
    .when(pl.col('Wind_Direction') == 'West').then(pl.lit('W'))
    .when(pl.col('Wind_Direction') == 'Variable').then(pl.lit('VAR'))
    .otherwise(pl.col('Wind_Direction'))  # Retain original value if no match
    .alias('Wind_Direction')  # Rename back to 'Wind_Direction'
)

In [37]:
unique_values = df_drop["Wind_Direction"].unique()
print(unique_values.to_list())

[None, 'SSW', 'CALM', 'NNW', 'NE', 'ENE', 'WSW', 'W', 'WNW', 'S', 'SSE', 'N', 'SE', 'NNE', 'ESE', 'VAR', 'NW', 'E', 'SW']


In [38]:
# Binning Time of Day
df_drop = df_drop.with_columns(
    pl.when(pl.col('Time_S').lt(6)).then(pl.lit('Early Morning'))
    .when(pl.col('Time_S').lt(12)).then(pl.lit('Morning'))
    .when(pl.col('Time_S').lt(18)).then(pl.lit('Afternoon'))
    .when(pl.col('Time_S').lt(24)).then(pl.lit('Evening'))
    .otherwise(pl.lit('Unknown')).alias('TimeofDay')
)

In [39]:
result = df_drop.filter(pl.col('TimeofDay') == 'Unknown')
print(result)

shape: (0, 32)
┌────────┬──────────┬───────────┬───────────┬───┬────────┬─────────┬──────────┬───────────┐
│ Source ┆ Severity ┆ Start_Lat ┆ Start_Lng ┆ … ┆ Time_S ┆ Weekday ┆ Duration ┆ TimeofDay │
│ ---    ┆ ---      ┆ ---       ┆ ---       ┆   ┆ ---    ┆ ---     ┆ ---      ┆ ---       │
│ str    ┆ i64      ┆ f64       ┆ f64       ┆   ┆ i8     ┆ i8      ┆ i64      ┆ str       │
╞════════╪══════════╪═══════════╪═══════════╪═══╪════════╪═════════╪══════════╪═══════════╡
└────────┴──────────┴───────────┴───────────┴───┴────────┴─────────┴──────────┴───────────┘


In [40]:
# Binning Seasons
df_drop = df_drop.with_columns(
    pl.when(pl.col('Month').is_in([12, 1, 2])).then(pl.lit('Winter'))
    .when(pl.col('Month').is_in([3, 4, 5])).then(pl.lit('Spring'))
    .when(pl.col('Month').is_in([6, 7, 8])).then(pl.lit('Summer'))
    .when(pl.col('Month').is_in([9, 10, 11])).then(pl.lit('Autumn'))
    .otherwise(pl.lit('Unknown')).alias('Season')
)

In [41]:
result = df_drop.filter(pl.col('Season') == 'Unknown')
print(result)

shape: (0, 33)
┌────────┬──────────┬───────────┬───────────┬───┬─────────┬──────────┬───────────┬────────┐
│ Source ┆ Severity ┆ Start_Lat ┆ Start_Lng ┆ … ┆ Weekday ┆ Duration ┆ TimeofDay ┆ Season │
│ ---    ┆ ---      ┆ ---       ┆ ---       ┆   ┆ ---     ┆ ---      ┆ ---       ┆ ---    │
│ str    ┆ i64      ┆ f64       ┆ f64       ┆   ┆ i8      ┆ i64      ┆ str       ┆ str    │
╞════════╪══════════╪═══════════╪═══════════╪═══╪═════════╪══════════╪═══════════╪════════╡
└────────┴──────────┴───────────┴───────────┴───┴─────────┴──────────┴───────────┴────────┘


In [42]:
df_drop.head(3)

Source,Severity,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Crossing,Junction,Traffic_Signal,Sunrise_Sunset,Year,Month,Day,Time_S,Weekday,Duration,TimeofDay,Season
str,i64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,str,i32,i32,i32,str,i32,i8,i8,i8,i8,i64,str,str
"""Source2""",3,39.865147,-84.058723,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US/Eastern""",36.9,58.251048,91.0,29.68,10.0,"""CALM""",7.68549,0.02,"""Light Rain""",0,0,0,"""Night""",2016,2,8,5,1,18840,"""Early Morning""","""Winter"""
"""Source2""",2,39.928059,-82.831184,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US/Eastern""",37.9,58.251048,100.0,29.65,10.0,"""CALM""",7.68549,0.0,"""Light Rain""",0,0,0,"""Night""",2016,2,8,6,1,1800,"""Morning""","""Winter"""
"""Source2""",2,39.063148,-84.032608,0.01,"""Accident on OH-32 State Route …","""State Route 32""","""Williamsburg""","""Clermont""","""OH""","""45176""","""US/Eastern""",36.0,33.3,100.0,29.67,10.0,"""SW""",3.5,0.008407,"""Overcast""",0,0,1,"""Night""",2016,2,8,6,1,1800,"""Morning""","""Winter"""


In [43]:
result = df_drop.filter(pl.col('Weekday') == 7)
print(result)

shape: (546_184, 33)
┌─────────┬──────────┬───────────┬─────────────┬───┬─────────┬──────────┬───────────┬────────┐
│ Source  ┆ Severity ┆ Start_Lat ┆ Start_Lng   ┆ … ┆ Weekday ┆ Duration ┆ TimeofDay ┆ Season │
│ ---     ┆ ---      ┆ ---       ┆ ---         ┆   ┆ ---     ┆ ---      ┆ ---       ┆ ---    │
│ str     ┆ i64      ┆ f64       ┆ f64         ┆   ┆ i8      ┆ i64      ┆ str       ┆ str    │
╞═════════╪══════════╪═══════════╪═════════════╪═══╪═════════╪══════════╪═══════════╪════════╡
│ Source2 ┆ 3        ┆ 39.981709 ┆ -82.98436   ┆ … ┆ 7       ┆ 49560    ┆ Morning   ┆ Winter │
│ Source2 ┆ 3        ┆ 39.953022 ┆ -82.998894  ┆ … ┆ 7       ┆ 1800     ┆ Morning   ┆ Winter │
│ Source2 ┆ 3        ┆ 39.973812 ┆ -82.983162  ┆ … ┆ 7       ┆ 1800     ┆ Morning   ┆ Winter │
│ Source2 ┆ 3        ┆ 39.978306 ┆ -82.852554  ┆ … ┆ 7       ┆ 4477     ┆ Evening   ┆ Winter │
│ Source2 ┆ 3        ┆ 39.917412 ┆ -83.014236  ┆ … ┆ 7       ┆ 4379     ┆ Evening   ┆ Winter │
│ …       ┆ …        ┆ …     

In [44]:
# Binning Day Type
df_drop = df_drop.with_columns(
    pl.when(pl.col('Weekday').is_in([1, 2, 3, 4])).then(pl.lit('Weekday'))
    .when(pl.col('Weekday').is_in([5, 6, 7])).then(pl.lit('Weekend'))
    .otherwise(pl.lit('Unknown')).alias('Day_Type')
)

In [45]:
unique_values = df_drop["Sunrise_Sunset"].unique()
print(unique_values.to_list())

['Day', None, 'Night']


In [46]:
df_drop = df_drop.with_columns(
    pl.when(pl.col('Sunrise_Sunset') == 'Night').then(0)
    .when(pl.col('Sunrise_Sunset') == 'Day').then(1)
    .otherwise(pl.col('Sunrise_Sunset'))  # Retain original value if no match
    .alias('Sunrise_Sunset')  # Rename back to 'Wind_Direction'
)

In [47]:
result = df_drop.filter(pl.col('Duration') <= 0)
print(result)

shape: (0, 34)
┌────────┬──────────┬───────────┬───────────┬───┬──────────┬───────────┬────────┬──────────┐
│ Source ┆ Severity ┆ Start_Lat ┆ Start_Lng ┆ … ┆ Duration ┆ TimeofDay ┆ Season ┆ Day_Type │
│ ---    ┆ ---      ┆ ---       ┆ ---       ┆   ┆ ---      ┆ ---       ┆ ---    ┆ ---      │
│ str    ┆ i64      ┆ f64       ┆ f64       ┆   ┆ i64      ┆ str       ┆ str    ┆ str      │
╞════════╪══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪════════╪══════════╡
└────────┴──────────┴───────────┴───────────┴───┴──────────┴───────────┴────────┴──────────┘


In [48]:
unique_values = df_drop["Weather_Condition"].unique()
print(unique_values.to_list())

['Showers in the Vicinity', 'Snow Grains', 'Light Rain', 'Cloudy / Windy', 'Wintry Mix', 'Light Rain Shower', 'Hail', 'Mist / Windy', 'Blowing Dust / Windy', 'Light Drizzle / Windy', 'Ice Pellets', 'Light Rain Shower / Windy', 'Drizzle and Fog', 'Heavy Ice Pellets', 'Widespread Dust', 'Thunder / Wintry Mix / Windy', 'Patches of Fog', 'Light Snow / Windy', 'Light Freezing Drizzle', 'Rain / Windy', 'Heavy Rain Shower', 'Heavy Freezing Rain', 'Thunder and Hail / Windy', 'Heavy Rain', 'Sand / Dust Whirlwinds / Windy', 'Fog', 'Thunder', 'Light Thunderstorms and Rain', 'Shallow Fog', 'Thunderstorms and Snow', 'Thunder and Hail', 'Heavy Rain Showers', 'Tornado', 'Drifting Snow', 'Thunder / Wintry Mix', 'Light Snow and Sleet', 'Light Snow Showers', 'Light Thunderstorms and Snow', 'Sleet and Thunder', 'Heavy Freezing Rain / Windy', 'Partial Fog', 'Light Snow with Thunder', 'Freezing Rain', 'Blowing Snow', 'Light Hail', 'Duststorm', 'Light Drizzle', 'Sand / Dust Whirlwinds', 'Widespread Dust / W

In [49]:
#We further clean wind direction by grouping similar values into one
df_drop = df_drop.with_columns(
    pl.when(pl.col('Wind_Direction') == 'Calm').then(pl.lit('CALM'))
    .when(pl.col('Wind_Direction').is_in(['West', 'WSW', 'WNW'])).then(pl.lit('W'))
    .when(pl.col('Wind_Direction').is_in(['South', 'SSW', 'SSE'])).then(pl.lit('S'))
    .when(pl.col('Wind_Direction').is_in(['North', 'NNW', 'NNE'])).then(pl.lit('N'))
    .when(pl.col('Wind_Direction').is_in(['East', 'ESE', 'ENE'])).then(pl.lit('E'))
    .when(pl.col('Wind_Direction') == 'Variable').then(pl.lit('VAR'))
    .otherwise(pl.col('Wind_Direction'))  # Retain original value if no match
    .alias('Wind_Direction')  # Rename back to 'Wind_Direction'
)

In [50]:
unique_values = df_drop["Wind_Direction"].unique()
print(unique_values.to_list())

['N', 'NW', 'SE', None, 'S', 'VAR', 'CALM', 'NE', 'SW', 'W', 'E']


In [51]:
df_drop.columns

['Source',
 'Severity',
 'Start_Lat',
 'Start_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Timezone',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Weather_Condition',
 'Crossing',
 'Junction',
 'Traffic_Signal',
 'Sunrise_Sunset',
 'Year',
 'Month',
 'Day',
 'Time_S',
 'Weekday',
 'Duration',
 'TimeofDay',
 'Season',
 'Day_Type']

In [52]:
# import plotly.express as px

# # Extract data as lists
# latitudes = df['Start_Lat'].to_list()
# longitudes = df['Start_Lng'].to_list()

# # Create a scatter mapbox plot with Plotly
# fig = px.scatter_mapbox(
#     lat=latitudes,
#     lon=longitudes,
#     zoom=3,
#     mapbox_style="open-street-map"
# )

# # Display the plot
# fig.show()

In [53]:
result = df_drop.filter(pl.col('Weather_Condition') == 'Volcanic Ash')
print(result)

shape: (22, 34)
┌─────────┬──────────┬───────────┬─────────────┬───┬──────────┬───────────┬────────┬──────────┐
│ Source  ┆ Severity ┆ Start_Lat ┆ Start_Lng   ┆ … ┆ Duration ┆ TimeofDay ┆ Season ┆ Day_Type │
│ ---     ┆ ---      ┆ ---       ┆ ---         ┆   ┆ ---      ┆ ---       ┆ ---    ┆ ---      │
│ str     ┆ i64      ┆ f64       ┆ f64         ┆   ┆ i64      ┆ str       ┆ str    ┆ str      │
╞═════════╪══════════╪═══════════╪═════════════╪═══╪══════════╪═══════════╪════════╪══════════╡
│ Source2 ┆ 2        ┆ 32.762203 ┆ -116.945045 ┆ … ┆ 2700     ┆ Afternoon ┆ Summer ┆ Weekday  │
│ Source2 ┆ 2        ┆ 32.838902 ┆ -116.934479 ┆ … ┆ 1791     ┆ Morning   ┆ Winter ┆ Weekend  │
│ Source2 ┆ 2        ┆ 32.825542 ┆ -116.90152  ┆ … ┆ 1782     ┆ Afternoon ┆ Winter ┆ Weekday  │
│ Source2 ┆ 2        ┆ 32.753311 ┆ -117.01152  ┆ … ┆ 1787     ┆ Morning   ┆ Winter ┆ Weekend  │
│ Source2 ┆ 3        ┆ 26.350828 ┆ -80.117874  ┆ … ┆ 1800     ┆ Morning   ┆ Summer ┆ Weekday  │
│ …       ┆ …        ┆ …

In [54]:
# Create weather categories
df_drop = df_drop.with_columns([
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("clear|fair")).then(1).otherwise(0).alias("Clear"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("cloud|overcast")).then(1).otherwise(0).alias("Cloud"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("rain|storm|drizzle|shower|precipitation|light hail|small hail")).then(1).otherwise(0).alias("Rain"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("heavy rain|rain shower|heavy t-storm|heavy thunderstorms|thunder|hail")).then(1).otherwise(0).alias("Heavy_Rain"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("snow|sleet|ice|wintry mix")).then(1).otherwise(0).alias("Snow"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("heavy snow|heavy sleet|heavy ice pellets|snow showers|squalls")).then(1).otherwise(0).alias("Heavy_Snow"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("fog|haze|mist")).then(1).otherwise(0).alias("Fog"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("dust|smoke|sand|tornado")).then(1).otherwise(0).alias("Dusty"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("wind|winds")).then(1).otherwise(0).alias("Windy"),
    pl.when(pl.col("Weather_Condition").str.to_lowercase().str.contains("ash")).then(1).otherwise(0).alias("Ash")
])

# Handle NA for the created weather features where 'Weather_Condition' is null
df_drop = df_drop.with_columns([
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Clear")).alias("Clear"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Cloud")).alias("Cloud"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Rain")).alias("Rain"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Heavy_Rain")).alias("Heavy_Rain"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Snow")).alias("Snow"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Heavy_Snow")).alias("Heavy_Snow"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Fog")).alias("Fog"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Dusty")).alias("Dusty"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Windy")).alias("Windy"),
    pl.when(pl.col("Weather_Condition").is_null()).then(None).otherwise(pl.col("Ash")).alias("Ash")
])

# Drop 'Weather_Condition' column
df_drop = df_drop.drop("Weather_Condition")

# Convert the new weather columns to integers
weather_columns = ['Clear', 'Cloud', 'Rain', 'Heavy_Rain', 'Snow', 'Heavy_Snow', 'Fog', 'Dusty', 'Windy', 'Ash']
df_drop = df_drop.with_columns([pl.col(w).cast(pl.Int32) for w in weather_columns])

In [55]:
df_drop.head(2)

Source,Severity,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Timezone,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Crossing,Junction,Traffic_Signal,Sunrise_Sunset,Year,Month,Day,Time_S,Weekday,Duration,TimeofDay,Season,Day_Type,Clear,Cloud,Rain,Heavy_Rain,Snow,Heavy_Snow,Fog,Dusty,Windy,Ash
str,i64,f64,f64,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,str,f64,f64,i32,i32,i32,str,i32,i8,i8,i8,i8,i64,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Source2""",3,39.865147,-84.058723,0.01,"""Right lane blocked due to acci…","""I-70 E""","""Dayton""","""Montgomery""","""OH""","""45424""","""US/Eastern""",36.9,58.251048,91.0,29.68,10.0,"""CALM""",7.68549,0.02,0,0,0,"""0""",2016,2,8,5,1,18840,"""Early Morning""","""Winter""","""Weekday""",0,0,1,0,0,0,0,0,0,0
"""Source2""",2,39.928059,-82.831184,0.01,"""Accident on Brice Rd at Tussin…","""Brice Rd""","""Reynoldsburg""","""Franklin""","""OH""","""43068-3402""","""US/Eastern""",37.9,58.251048,100.0,29.65,10.0,"""CALM""",7.68549,0.0,0,0,0,"""0""",2016,2,8,6,1,1800,"""Morning""","""Winter""","""Weekday""",0,0,1,0,0,0,0,0,0,0


In [56]:
df_drop.width

43

In [57]:
df_drop.columns

['Source',
 'Severity',
 'Start_Lat',
 'Start_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Timezone',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Crossing',
 'Junction',
 'Traffic_Signal',
 'Sunrise_Sunset',
 'Year',
 'Month',
 'Day',
 'Time_S',
 'Weekday',
 'Duration',
 'TimeofDay',
 'Season',
 'Day_Type',
 'Clear',
 'Cloud',
 'Rain',
 'Heavy_Rain',
 'Snow',
 'Heavy_Snow',
 'Fog',
 'Dusty',
 'Windy',
 'Ash']

In [58]:
df_drop = df_drop.drop(['Description', 'Street','Zipcode','Source','Year', 'Month', 'Day', 'Time_S', 'Weekday'])
#Description, Street and Zipcode are irrelevant features in severity analysis
#Source indicates from which source the data is captured, which is not useful again
#Year,Month,Day,Time_S and Weekday were used to create TimeofDay,Season,Day_Type through binning which are more useful features,hence these are removed

In [59]:
df_drop.width

34

In [60]:
used_features=['Severity', 'Distance(mi)','Temperature(F)','Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)','Wind_Direction',
               'Wind_Speed(mph)','Precipitation(in)','Clear','Cloud','Rain','Heavy_Rain','Snow','Heavy_Snow','Fog', 'Dusty', 'Windy','Ash','Junction',
               'Crossing','Traffic_Signal','Sunrise_Sunset','TimeofDay', 'Season', 'Day_Type', 'Duration'] 
print(len(used_features))

28
